In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Importations
import seaborn as sns
import torch

from torch.utils.data import DataLoader, BatchSampler, random_split

from src.covid_cytof_dataset import CovidCytofDataset
from src.models.simple_nn import SimpleNeuralNetwork
from src.trainer import Trainer
from itertools import chain
from torch import nn

sns.set_theme()

In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
device = "cpu"
print(f"Using {device} device")

In [ ]:
BATCH_SIZE = 10000

In [ ]:
data = CovidCytofDataset("./data/attachments/COVID_CYTOF_BASIC_METADATA_STATUS_AGE_GROUP_SEX.xlsx", "./data", BATCH_SIZE)

In [ ]:
train_data, test_data = random_split(list(BatchSampler(data, batch_size=BATCH_SIZE, drop_last=False)), [0.8, 0.2])
train_data, test_data = list(chain.from_iterable(train_data)), list(chain.from_iterable(test_data))

train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

# del train_data, test_data

In [ ]:
model = SimpleNeuralNetwork(data.data.shape[1]).to(device)
print(model)

In [ ]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

trainer = Trainer(model, optimizer, loss_fn, device, epochs=10)

In [ ]:
trainer.run(train_dataloader, test_dataloader)

In [ ]:
trainer.plot_loss()
trainer.plot_accuracy()